In [ ]:
# Import csv file using the pandas library--------------------------------------------------------------------------------------
import sqlite3
import pandas as pd
from datetime import datetime

# Read Orders CSV file into pandas dataframe
Orders_df = pd.read_csv('Orders.csv')

# Clean the orders dataframe from missing values by filling specific missing values with given values
Orders_df.iloc[186,0] = "OD187"
Orders_df.iloc[335,1] = "Charlie"
Orders_df.iloc[381,6] = "Cash on Delivery"
Orders_df.iloc[486,6] = "Credit Card"
Orders_df.iloc[492,6] = "Credit Card"
Orders_df = Orders_df.fillna(0)


# Read restaurant_info CSV file into pandas dataframe
restaurant_info_df = pd.read_csv('restaurant_info.csv')

# Clean the restaurant_info dataframe from missing values by filling specific missing values with given values
restaurant_info_df.iloc[7,6] = "Adeleke Daniel"
restaurant_info_df.iloc[7,7] = 5
restaurant_info_df.iloc[7,8] = "dannydave1000@gmail.com"
restaurant_info_df.iloc[14,7] = 7
restaurant_info_df.iloc[16,8] = "lekeman1000@gmail.com"
restaurant_info_df = restaurant_info_df.fillna(0)


# Merge Orders_df with restaurant_info_df on the common column 'RestaurantName'
merged_df = pd.merge(Orders_df, restaurant_info_df, how='inner', left_on='Restaurant ID', right_on='RestaurantID')

# Connect to SQLite database
conn = sqlite3.connect('restaurant_orders.db')
conn.execute("PRAGMA foreign_keys = ON")
cursor = conn.cursor()

# Create Manager table----------------------------------------------------------------------------------------------------------
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Manager (
        ManagerID INTEGER PRIMARY KEY AUTOINCREMENT,
        ManagerName VARCHAR(255) NOT NULL,
        YearsAsManager INT NOT NULL,
        Email VARCHAR(255) NOT NULL
    );
""")

# Create DeliveryStaff table-----------------------------------------------------------------------------------------------------
cursor.execute("""
    CREATE TABLE IF NOT EXISTS DeliveryStaff (
        StaffID INTEGER PRIMARY KEY AUTOINCREMENT,
        FirstName VARCHAR(255) NOT NULL,
        LastName VARCHAR(255) NOT NULL,
        VehicleType VARCHAR(255) NOT NULL
    );
""")

# Create Restaurant table-------------------------------------------------------------------------------------------------------
cursor.execute("""
    CREATE TABLE IF NOT EXISTS restaurant_info (
        RestaurantID INTEGER PRIMARY KEY AUTOINCREMENT,
        RestaurantName VARCHAR(255) NOT NULL,
        Cuisine VARCHAR(255) NOT NULL,
        Zone VARCHAR(255) NOT NULL,
        Category VARCHAR(255) NOT NULL,
        Store INT NOT NULL,
        Address VARCHAR(255) NOT NULL,
        ManagerID INTEGER NOT NULL,
        StaffID INTEGER NOT NULL,
        FOREIGN KEY (ManagerID) REFERENCES Manager(ManagerID),
        FOREIGN KEY (StaffID) REFERENCES DeliveryStaff(StaffID)
    );
""")


# Create Customers table---------------------------------------------------------------------------------------------------------
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Customers (
        CustomerID INTEGER PRIMARY KEY AUTOINCREMENT,
        First_Customer_Name VARCHAR(255) NOT NULL,
        Last_Customer_Name VARCHAR(255) NOT NULL
    );
""")     
    
# Create Payment_info------------------------------------------------------------------------------------------------------------
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Payment_info (
        PaymentID INTEGER PRIMARY KEY AUTOINCREMENT,
        PaymentMode VARCHAR(255) NOT NULL,
        DebitCard REAL NOT NULL,
        CreditCard REAL NOT NULL,
        CardProvider VARCHAR(255) NOT NULL
    );
""")

# Create Orders table-----------------------------------------------------------------------------------------------------------
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Orders (
        ID INTEGER PRIMARY KEY AUTOINCREMENT,
        OrderID VARCHAR(255) NOT NULL,
        OrderDate DATETIME NOT NULL,
        Quantity_of_Items INT,
        OrderAmount INT NOT NULL,
        DeliveryTimeTaken INT NOT NULL,
        CustomerRatingFood INT NOT NULL,
        CustomerRatingDelivery INT NOT NULL,
        RestaurantsID INTEGER NOT NULL,
        PaymentID INTEGER NOT NULL, 
        StaffID INTEGER NOT NULL,
        CustomerID INTEGER NOT NULL,
        FOREIGN KEY (RestaurantsID) REFERENCES restaurant_info(RestaurantID),
        FOREIGN KEY (PaymentID) REFERENCES Payment_info(PaymentID),
        FOREIGN KEY (StaffID) REFERENCES DeliveryStaff(StaffID)
        FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
    );
""")


# Insert data into tables----------------------------------------------------------------------------------------------------------

# Insert data into DeliveryStaff table
delivery_staff_data = [
    ("John", "Doe", "Bike"),
    ("Jane", "Smith", "Car"),
    ("Dave", "Mark", "Bike"),
    ("Bill", "Huber", "Motorbike"),
    ("Aryna", "Sabalenka", "Car"),
    ("Gerlad", "Hold", "Car"),
    ("Adam", "Smith", "Motorbike"),
    ("Janet", "Rajesh", "Motorbike"),
    ("Isaac", "Shaw", "Car"),
    ("Benedict", "Fall", "Bike"),
    ("Andrew", "Pears", "Bike"),
    ("William", "Hive", "Bike"),
    ("Fred", "Short", "Car"),
    ("Helena", "Gates", "Motorbike"),
    ("Gaius", "Finn", "Motorbike"),
    ("Thomas", "Fire", "Car"),
    ("Vivian", "Geld", "Car"),
    ("Lucy", "Tor", "Car"),
    ("James", "Gold", "Bike"),
    ("David", "Sean", "Motorbike"),
]

for staff_record in delivery_staff_data:
    cursor.execute('''
        INSERT INTO DeliveryStaff (FirstName, LastName, VehicleType)
        VALUES (?, ?, ?)
    ''', staff_record)
    cursor.execute('SELECT last_insert_rowid()')
    staff_id = cursor.fetchone()[0]
    

# Insert data into Customers table
customers_data = [
    ("Srini", "Simon"),
    ("Revandh", "Scott"),
    ("David", "Torres"),
    ("Selva", "Jones"),
    ("Vinny", "Coleman"),
    ("Dev", "Armstrong"),
    ("Meera", "Gates"),
    ("Sabeena", "Allen"),
    ("Rifa", "Carney"),
    ("Farhan", "Gillespie"),
    ("Suhaib", "Henry"),
    ("Swamy", "Evans"),
    ("Sweetie", "Nguyen"),
    ("Shifa", "Anderson"),
    ("Charlie", "Brooks"),
    ("Veer", "Martinez"),
    ("Chinny", "Ferguson"),
    ("Gopal", "Nelson"),
    ("Veronica", "Russell"),
    ("Fastin", "Smith"),
    ("Ram", "Martin"),
]

for customer_record in customers_data:
    cursor.execute('''
        INSERT INTO Customers (First_Customer_Name, Last_Customer_Name)
        VALUES (?, ?)
    ''', customer_record)
    cursor.execute('SELECT last_insert_rowid()')
    customers_id = cursor.fetchone()[0]
    
# Insert data into Manager table
for index, row in restaurant_info_df.iterrows():
    cursor.execute('''
        INSERT INTO Manager (ManagerName, YearsAsManager, Email)
        VALUES (?, ?, ?)
    ''', (row['Manager'], row['Years_as_manager'], row['Email'],))
    cursor.execute('SELECT last_insert_rowid()')
    manager_id = cursor.fetchone()[0]
    
# Insert data into restaurant_info table
for index, row in restaurant_info_df.iterrows():
    cursor.execute('''
        INSERT INTO restaurant_info (RestaurantName, Cuisine, Zone, Category, Store, Address, ManagerID, StaffID)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (row['RestaurantName'], row['Cuisine'], row['Zone'], row['Category'], row['Store'], row['Address'], manager_id, staff_id))
    cursor.execute('SELECT last_insert_rowid()')
    restaurants_id = cursor.fetchone()[0]

# Insert Payment info into Payment_info table
for index, row in merged_df.iterrows():
    cursor.execute('''
        INSERT INTO Payment_info (PaymentMode, DebitCard, CreditCard, CardProvider)
        VALUES (?, ?, ?, ?)
    ''', (row['Payment Mode'], row['Debit Card'], row['Credit Card'], row['Card provider'],))
    cursor.execute('SELECT last_insert_rowid()')
    payment_id = cursor.fetchone()[0]
    
# Convert 'Order Date' to datetime and sort DataFrame
merged_df['Order Date'] = pd.to_datetime(merged_df['Order Date'])
new_merged_df = merged_df.sort_values(by='Order Date')

# Insert data into Orders table
for index, row in new_merged_df.iterrows():
    order_date_str = row['Order Date'].strftime('%Y-%m-%d %H:%M:%S')

    cursor.execute('''
        INSERT INTO Orders (OrderID, OrderDate, Quantity_of_Items, OrderAmount, DeliveryTimeTaken, CustomerRatingFood, CustomerRatingDelivery, RestaurantsID, PaymentID, StaffID, CustomerID)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (row['Order ID'], order_date_str, row['Quantity of Items'], row['Order Amount'], row['Delivery Time Taken (mins)'], row['Customer Rating-Food'], row['Customer Rating-Delivery'], row['RestaurantID'], payment_id, staff_id, customers_id,))
    
    
for index, row in merged_df.iterrows():
    cursor.execute("""
        SELECT CustomerID
        FROM Customers
        WHERE First_Customer_Name = ? AND Last_Customer_Name = ?;
    """, (row['First_Customer_Name'], row['Last_Customer_Name'],))
    result = cursor.fetchone()
    if result:
        customer_id = result[0]
        cursor.execute("""
            UPDATE Orders
            SET CustomerID = ?
            WHERE OrderID = ?;
        """, (customer_id, row['Order ID']))
        
        
for index, row in merged_df.iterrows():
    cursor.execute("""
        SELECT ManagerID
        FROM Manager
        WHERE ManagerName = ?;
    """, (row['Manager'],))
    result = cursor.fetchone()

    if result:
        manager_id = result[0]
        cursor.execute("""
            UPDATE restaurant_info
            SET ManagerID = ?
            WHERE RestaurantID = ?;
        """, (manager_id, row['RestaurantID']))
        
for index, row in merged_df.iterrows():
    cursor.execute("""
        SELECT PaymentID
        FROM Payment_info
        WHERE PaymentMode = ?;
    """, (row['Payment Mode'],))
    result = cursor.fetchone()

    if result:
        payment_id = result[0]
        cursor.execute("""
            UPDATE Orders
            SET PaymentID = ?
            WHERE OrderID = ?;
        """, (payment_id, row['Order ID']))
        
    
# Commit and close the connection
conn.commit()
conn.close()
print("Database created and data loaded successfully.")

In [ ]:
# GUI development with tkinter
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
import re
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from tkinter import simpledialog

# Functions to fetch data from the database and display it----------------------------------------------------------------------------
def fetch_restaurant_info():
    tree_restaurant_info.delete(*tree_restaurant_info.get_children())
    connection = sqlite3.connect("restaurant_orders.db")
    cursor = connection.cursor()
    cursor.execute("PRAGMA table_info(restaurant_info)")  # Get column names
    columns = [column[1] for column in cursor.fetchall()]
    
    tree_restaurant_info.insert('', tk.END, values=columns, tags=('header',))
    
    cursor.execute("SELECT * FROM restaurant_info")
    records = cursor.fetchall()
    for record in records:
        tree_restaurant_info.insert('', tk.END, values=record)
    cursor.close()
    connection.close()

def fetch_orders():
    tree_orders.delete(*tree_orders.get_children())
    connection = sqlite3.connect("restaurant_orders.db")
    cursor = connection.cursor()
    cursor.execute("PRAGMA table_info(Orders)")  # Get column names
    columns = [column[1] for column in cursor.fetchall()]

    # Insert column titles
    tree_orders.insert('', tk.END, values=columns, tags=('header',))
    
    cursor.execute("SELECT * FROM Orders")
    records = cursor.fetchall()
    for record in records:
        tree_orders.insert('', tk.END, values=record)
    cursor.close()
    connection.close()
    
def fetch_Manager():
    tree_manager.delete(*tree_manager.get_children())  # Assuming tree_manager is your Treeview for the Manager tab
    connection = sqlite3.connect("restaurant_orders.db")
    cursor = connection.cursor()
    cursor.execute("PRAGMA table_info(Manager)")  # Get column names
    columns = [column[1] for column in cursor.fetchall()]

    # Insert column titles
    tree_manager.insert('', tk.END, values=columns, tags=('header',))

    cursor.execute("SELECT * FROM Manager")
    records = cursor.fetchall()
    for record in records:
        tree_manager.insert('', tk.END, values=record)
    cursor.close()
    connection.close()

def fetch_Payment_info():
    tree_payment_info.delete(*tree_payment_info.get_children())  # Assuming tree_payment_info is your Treeview for the Payment_info tab
    connection = sqlite3.connect("restaurant_orders.db")
    cursor = connection.cursor()
    
    cursor.execute("PRAGMA table_info(Payment_info)")  # Get column names
    columns = [column[1] for column in cursor.fetchall()]

    # Insert column titles
    tree_payment_info.insert('', tk.END, values=columns, tags=('header',))

    cursor.execute("SELECT * FROM Payment_info")
    records = cursor.fetchall()
    for record in records:
        tree_payment_info.insert('', tk.END, values=record)
    cursor.close()
    connection.close()
    
def fetch_Customers():
    tree_customers.delete(*tree_customers.get_children())
    connection = sqlite3.connect("restaurant_orders.db")
    cursor = connection.cursor()
    cursor.execute("PRAGMA table_info(Customers)")  # Get column names
    columns = [column[1] for column in cursor.fetchall()]

    # Insert column titles
    tree_customers.insert('', tk.END, values=columns, tags=('header',))

    cursor.execute("SELECT * FROM Customers")
    records = cursor.fetchall()
    for record in records:
        tree_customers.insert('', tk.END, values=record)
    cursor.close()
    connection.close()

def fetch_DeliveryStaff():
    tree_DeliveryStaff.delete(*tree_DeliveryStaff.get_children())
    connection = sqlite3.connect("restaurant_orders.db")
    cursor = connection.cursor()
    cursor.execute("PRAGMA table_info(DeliveryStaff)")  # Get column names
    columns = [column[1] for column in cursor.fetchall()]

    # Insert column titles
    tree_DeliveryStaff.insert('', tk.END, values=columns, tags=('header',))

    cursor.execute("SELECT * FROM DeliveryStaff")
    records = cursor.fetchall()
    for record in records:
        tree_DeliveryStaff.insert('', tk.END, values=record)
    cursor.close()
    connection.close()


# Function to validate the email format-----------------------------------------------------------------------------------------
def validate_email(Email):
    try:
        # Use a simple regex to check email format
        if not pd.isna(Email) and pd.notna(Email):
            return bool(re.match(r"[^@]+@[^@]+\.[^@]+", Email))
        return False
    except Exception as e:
        print(f"Error validating email: {e}")
        return False

# Function to update the manager table in the restaurant_orders database---------------------------------------------------------------------------------
def update_manager():
    manager_id = entry_Managerid.get()
    manager_name = entry_ManagerName.get()
    years_as_manager = entry_YearAsManager.get()
    email = entry_Email.get()
    

    if not validate_email(email):
        messagebox.showerror("Error", "Invalid email format (must be ccc@aaa.bbb).")
        return

    try:
        conn = sqlite3.connect('restaurant_orders.db')
        cursor = conn.cursor()
        cursor.execute("""
        UPDATE Manager
        SET ManagerName = ?, YearsAsManager = ?, Email = ?
        WHERE ManagerID = ?
        """, (manager_name, years_as_manager, email, manager_id))
        conn.commit()
        messagebox.showinfo("Success", "Manager updated successfully.")
    except Exception as e:
        messagebox.showerror("Error", str(e))
    finally:
        conn.close()

# Function to calculate and show the mean customer rating for food-----------------------------------------------------------------------------
def calculate_mean_rating():
    try:
        conn = sqlite3.connect('restaurant_orders.db')
        cursor = conn.cursor()
        cursor.execute("SELECT AVG([CustomerRatingFood]) FROM Orders")
        mean_rating = cursor.fetchone()[0]
        conn.close()

        messagebox.showinfo("Mean Customer Rating", f"Mean Customer Rating for Food: {mean_rating:.2f}")
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Function to draw a histogram of delivery time taken-----------------------------------------------------------------------------------
def draw_histogram():
    try:
        conn = sqlite3.connect('restaurant_orders.db')
        delivery_times = pd.read_sql_query("SELECT [DeliveryTimeTaken] FROM Orders", conn)
        plt.figure()
        plt.hist(delivery_times['DeliveryTimeTaken'], bins=20, color='#e856c6', edgecolor='black')
        plt.title("Histogram of Delivery Time Taken (mins)")
        plt.xlabel("Time (mins)")
        plt.ylabel("Number of Orders")

        # Display the histogram within the GUI
        fig = plt.gcf()
        canvas = FigureCanvasTkAgg(fig, master=tab_statistics)
        canvas_widget = canvas.get_tk_widget()
        canvas_widget.pack()

    except Exception as e:
        messagebox.showerror("Error", str(e))
    finally:
        conn.close()
        
        
# Function to define delete selected--------------------------------------------------------------------------------------------
def delete_selected_(tree, restaurant_info):
    selected_item = tree.selection()
    if selected_item:
        conn = sqlite3.connect('restaurant_orders.db')
        cursor = conn.cursor()
        cursor.execute(f"DELETE FROM {restaurant_info} WHERE RestaurantID=?", (selected_item[0],))
        conn.commit()
        conn.close()
        print(f"Deleted order {selected_item[0]}...")
        refresh_table(tree, restaurant_info)
        
def delete_selected(tree, Orders):
    selected_item = tree.selection()
    if selected_item:
        conn = sqlite3.connect('restaurant_orders.db')
        cursor = conn.cursor()
        cursor.execute(f"DELETE FROM {Orders} WHERE ID=?", (selected_item[0],))
        conn.commit()
        conn.close()
        print(f"Deleted order {selected_item[0]}...")
        refresh_table(tree, Orders)

# Function to define update info------------------------------------------------------------------------------------------------------
def update_info_(tree, restaurant_info):
    selected_item = tree.selection()
    if selected_item:
        conn = sqlite3.connect('restaurant_orders.db')
        cursor = conn.cursor()
        # Example: Use a dialog to get the updated name
        new_name = simpledialog.askstring("Update Info", "Enter new name:")
        cursor.execute(f"UPDATE {restaurant_info} SET RestaurantName=? WHERE RestaurantID=?", (new_name, selected_item[0]))
        conn.commit()
        conn.close()
        print(f"Updated info for Restaurant info {selected_item[0]}...")
        refresh_table(tree, restaurant_info)
        
def update_info(tree, Orders):
    selected_item = tree.selection()
    if selected_item:
        conn = sqlite3.connect('restaurant_orders.db')
        cursor = conn.cursor()
        # Example: Use a dialog to get the updated name
        new_name = simpledialog.askstring("Update Info", "Enter new name:")
        cursor.execute(f"UPDATE {Orders} SET First_Customer_Name=? WHERE ID=?", (new_name, selected_item[0]))
        conn.commit()
        conn.close()
        print(f"Updated info for order {selected_item[0]}...")
        refresh_table(tree, Orders)

# Function to define new info-------------------------------------------------------------------------------------------------------
def new_info_(tree, restaurant_info):
    conn = sqlite3.connect('restaurant_orders.db')
    cursor = conn.cursor()
    # Use dialogs to get new information
    new_restaurant_name = simpledialog.askstring("New Info", "Enter Restaurant name:")
    new_cuisine = simpledialog.askstring("New Info", "Enter Cuisine name:")
    new_zone = simpledialog.askstring("New Info", "Enter Zone name:")
    new_category = simpledialog.askstring("New Info", "Enter Category name:")
    new_store = simpledialog.askinteger("New Info", "Enter Store number:")
    new_address = simpledialog.askstring("New Info", "Enter Address:")
    
    cursor.execute(f"INSERT INTO {restaurant_info} (RestaurantName, Cuisine, Zone, Category, Store, Address) VALUES (?, ?, ?, ?, ?, ?)", (new_restaurant_name, new_cuisine, new_zone, new_category, new_store, new_address))
    conn.commit()
    conn.close()
    print("Added new restaurant info...")

def new_info(tree, Orders):
    conn = sqlite3.connect('restaurant_orders.db')
    cursor = conn.cursor()
    # Use dialogs to get new information
    new_Order_id = simpledialog.askstring("New Info", "Enter Order ID:")
    new_order_date = simpledialog.askstring("New Info", "Enter Order Date:")
    new_quantity = simpledialog.askstring("New Info", "Enter Quantity:")
    new_order_amount = simpledialog.askinteger("New Info", "Enter Order Amount:")
    new_delivery_time_taken = simpledialog.askinteger("New Info", "Enter Delivery Time Taken '(mins)':")
    new_customer_rating_food = simpledialog.askinteger("New Info", "Enter Customer Rating-Food:")
    new_customer_rating_delivery = simpledialog.askinteger("New Info", "Enter Customer Rating-delivery:")    
    
    cursor.execute(f"INSERT INTO {Orders} (OrderID, OrderDate, Quantity_of_Items, OrderAmount, DeliveryTimeTaken, CustomerRatingFood, CustomerRatingDelivery) VALUES (?, ?, ?, ?, ?, ?, ?)", (new_Order_id, new_order_date, new_quantity, new_order_amount, new_delivery_time_taken, new_customer_rating_food, new_customer_rating_delivery,))
    conn.commit()
    conn.close()
    print("Added new order info...")
    

app = tk.Tk()
app.title("Restaurant and Order Management")
app.geometry('1200x1000')

# Style configuration
style = ttk.Style()
style.configure("TNotebook", tabposition="nw")
style.configure("TButton", padding=(10, 5), width=20)
style.theme_use('clam') 
style.configure('TLabel', background='#DCDAD5', font=('Arial', 10))
style.configure('TEntry', padding=5)
style.configure('TButton', font=('Arial', 10), background='#fc8eac')

# Create the notebook
notebook = ttk.Notebook(app)
notebook.pack(expand=True, fill='both')

# Create the tabs
tab_manager_update = ttk.Frame(notebook)
tab_restaurant_info = ttk.Frame(notebook)
tab_orders = ttk.Frame(notebook)
tab_manager = ttk.Frame(notebook)
tab_payment_info = ttk.Frame(notebook)
tab_customers = ttk.Frame(notebook)
tab_deliverystaff = ttk.Frame(notebook)
tab_statistics = ttk.Frame(notebook)

# Add tabs to the notebook
notebook.add(tab_manager_update, text='Manager Update')
notebook.add(tab_restaurant_info, text='Restaurant Info Table')
notebook.add(tab_orders, text='Orders Table')
notebook.add(tab_manager, text='Manager Table')
notebook.add(tab_payment_info, text='Payment Info Table ')
notebook.add(tab_customers, text='Customers Table')
notebook.add(tab_deliverystaff, text='Delivery Staff Table')
notebook.add(tab_statistics, text='Statistics')

# Create header frames for each tab with a title-------------------------------------------------------------------------------------
header_manager = ttk.Label(tab_manager_update, text="Manager Information Update", font=("Arial", 25))
header_manager.pack(side=tk.TOP, pady=(5, 10))

header_restaurant_info = ttk.Label(tab_restaurant_info, text="Restaurant Info Table", font=("Arial", 25))
header_restaurant_info.pack(side=tk.TOP, pady=(5, 10))

header_orders = ttk.Label(tab_orders, text="Orders Table", font=("Arial", 25))
header_orders.pack(side=tk.TOP, pady=(5, 10))

header_manager = ttk.Label(tab_manager, text="Manager Table", font=("Arial", 25))
header_manager.pack(side=tk.TOP, pady=(5, 10))

header_payment_info = ttk.Label(tab_payment_info, text="Payment Info Table", font=("Arial", 25))
header_payment_info.pack(side=tk.TOP, pady=(5, 10))

header_customers = ttk.Label(tab_customers, text="Customers Table", font=("Arial", 25))
header_customers.pack(side=tk.TOP, pady=(5, 10))

header_deliverystaff = ttk.Label(tab_deliverystaff, text="Delivery Staff Table", font=("Arial", 25))
header_deliverystaff.pack(side=tk.TOP, pady=(5, 10))

header_statistics = ttk.Label(tab_statistics, text="Statistics and Analysis", font=("Arial", 25))
header_statistics.pack(side=tk.TOP, pady=(5, 10))


# Manager_update tab------------------------------------------------------------------------------------------------------------
manager_frame = ttk.LabelFrame(tab_manager_update, text="Manager Information Update")
manager_frame.pack(padx=10, pady=10, fill="both", expand=True)
manager_frame.columnconfigure(0, weight=1)
manager_frame.columnconfigure(1, weight=2)
entry_width = 30
label_width = 20

manager_id_label = ttk.Label(manager_frame, text="ID:", width=label_width, anchor='e')
manager_id_label.grid(row=0, column=0, padx=5, pady=2, sticky='ew')
entry_Managerid = ttk.Entry(manager_frame, width=entry_width)
entry_Managerid.grid(row=0, column=1, padx=5, pady=18, sticky='ew')

manager_name_label = ttk.Label(manager_frame, text="Manager Name:", width=label_width, anchor='e')
manager_name_label.grid(row=1, column=0, padx=5, pady=2, sticky='ew')
entry_ManagerName = ttk.Entry(manager_frame, width=entry_width)
entry_ManagerName.grid(row=1, column=1, padx=5, pady=18, sticky='ew')

year_as_manager_label = ttk.Label(manager_frame, text="Year As Manager:", width=label_width, anchor='e')
year_as_manager_label.grid(row=2, column=0, padx=5, pady=2, sticky='ew')
entry_YearAsManager = ttk.Entry(manager_frame, width=entry_width)
entry_YearAsManager.grid(row=2, column=1, padx=5, pady=18, sticky='ew')

manager_email_label = ttk.Label(manager_frame, text="Manager Email:", width=label_width, anchor='e')
manager_email_label.grid(row=3, column=0, padx=5, pady=2, sticky='ew')
entry_Email = ttk.Entry(manager_frame, width=entry_width)
entry_Email.grid(row=3, column=1, padx=5, pady=18, sticky='ew')

update_manager_button = ttk.Button(manager_frame, text="Update Manager", command=update_manager)
update_manager_button.grid(row=4, column=0, columnspan=2, pady=10, sticky='ew')

manager_frame.grid_rowconfigure(5, weight=1)

# Restaurant Info tab------------------------------------------------------------------------------------------------------------
info_frame = ttk.LabelFrame(tab_restaurant_info, text="Restaurant Info Table")
info_frame.columnconfigure(0, weight=1)
info_frame.rowconfigure(0, weight=1)
info_frame.pack(padx=10, pady=10, fill='both', expand=True)

tree_restaurant_info = ttk.Treeview(info_frame, columns=('RestaurantID', 'RestaurantName', 'Cuisine', 'Zone', 'Category', 'Store', 'Address', 'ManagerID'))
tree_restaurant_info.pack(padx=10, pady=10)

columns_order = ('RestaurantID', 'RestaurantName', 'Cuisine', 'Zone', 'Category', 'Store', 'Address', 'ManagerID')

fetch_restaurant_info_button = tk.Button(info_frame, text="Display Restaurant Info", command=fetch_restaurant_info)
fetch_restaurant_info_button.pack(pady=10)

delete_restaurant_info_button = tk.Button(info_frame, text="Delete Restaurant Info", command=lambda: delete_selected_(tree_restaurant_info, 'restaurant_info'))
delete_restaurant_info_button.pack(pady=10)

update_restaurant_info_button = tk.Button(info_frame, text="Update Restaurant Info", command=lambda: update_info_(tree_restaurant_info, 'restaurant_info'))
update_restaurant_info_button.pack(pady=10)

new_restaurant_info_button = tk.Button(info_frame, text="Add New Restaurant Info", command=lambda: new_info_(tree_restaurant_info, 'restaurant_info'))
new_restaurant_info_button.pack(pady=10)

# Orders tab---------------------------------------------------------------------------------------------------------------------
orders_frame = ttk.LabelFrame(tab_orders, text="Orders Table")
orders_frame.columnconfigure(0, weight=1)
orders_frame.rowconfigure(0, weight=1)
orders_frame.pack(padx=10, pady=10, fill='both', expand=True)

tree_orders = ttk.Treeview(orders_frame, columns=('ID','OrderID', 'Order Date', 'Quantity_of_Items', 'Order Amount', 'Delivery Time Taken ', 'Customer Rating Food', 'Customer Rating Delivery', 'RestaurantsID', 'PaymentID', 'CustomerID', 'StaffID'))
tree_orders.pack(padx=10, pady=10)

columns_order = ('ID','OrderID', 'Order Date', 'Quantity_of_Items', 'OrderAmount', 'DeliveryTimeTaken', 'CustomerRatingFood', 'CustomerRatingDelivery', 'RestaurantsID', 'PaymentID', 'CustomerID', 'StaffID')

fetch_orders_button = tk.Button(orders_frame, text="Display Orders", command=fetch_orders)
fetch_orders_button.pack(pady=10)

delete_orders_button = tk.Button(orders_frame, text="Delete Orders", command=lambda: delete_selected(tree_orders, 'orders'))
delete_orders_button.pack(pady=10)

update_orders_button = tk.Button(orders_frame, text="Update Orders", command=lambda: update_info(tree_orders, 'orders'))
update_orders_button.pack(pady=10)

new_orders_button = tk.Button(orders_frame, text="Add New Orders", command=lambda: new_info(tree_orders, 'orders'))
new_orders_button.pack(pady=10)

# Manager tab----------------------------------------------------------------------------------------------------------------------
manager_frame = ttk.LabelFrame(tab_manager, text="Manager Table")
manager_frame.columnconfigure(0, weight=1)
manager_frame.rowconfigure(0, weight=1)
manager_frame.pack(padx=10, pady=10, fill='both', expand=True)

tree_manager = ttk.Treeview(manager_frame, columns=('ManagerID','Manager Name', 'Year As Manager', 'Email'))
tree_manager.pack(padx=10, pady=10)

columns_manager = ('ManagerID', 'ManagerName', 'YearsAsManager', 'Email')

fetch_manager_button = tk.Button(manager_frame, text="Display", command=fetch_Manager)
fetch_manager_button.pack(pady=10)

# Payment_info tab-----------------------------------------------------------------------------------------------------------------
payment_info_frame = ttk.LabelFrame(tab_payment_info, text="Payment Info Table")
payment_info_frame.columnconfigure(0, weight=1)
payment_info_frame.rowconfigure(0, weight=1)
payment_info_frame.pack(padx=10, pady=10, fill='both', expand=True)

tree_payment_info = ttk.Treeview(payment_info_frame, columns=('PaymentID', 'Payment Method', 'Amount', 'OrderID', 'CustomerID'))
tree_payment_info.pack(padx=10, pady=10)

columns_payment_info = ('PaymentID', 'PaymentMethod', 'Amount', 'OrderID', 'CustomerID')

fetch_payment_info_button = tk.Button(payment_info_frame, text="Display", command=fetch_Payment_info)
fetch_payment_info_button.pack(pady=10)

# Customers tab
customers_frame = ttk.LabelFrame(tab_customers, text="Customers Table")
customers_frame.columnconfigure(0, weight=1)
customers_frame.rowconfigure(0, weight=1)
customers_frame.pack(padx=10, pady=10, fill='both', expand=True)

tree_customers = ttk.Treeview(customers_frame, columns=('CustomerID', 'First Customer Name', 'Last Customer Name'))
tree_customers.pack(padx=10, pady=10)

columns_payment_info = ('CustomerID', 'First_Customer_Name', 'Last_Customer_Name',)

fetch_customers_button = tk.Button(customers_frame, text="Display", command=fetch_Customers)
fetch_customers_button.pack(pady=10)

# Delivery Staff tab----------------------------------------------------------------------------------------------------------------
deliverystaff_frame = ttk.LabelFrame(tab_deliverystaff, text="Delivery Staff Table")
deliverystaff_frame.columnconfigure(0, weight=1)
deliverystaff_frame.rowconfigure(0, weight=1)
deliverystaff_frame.pack(padx=10, pady=10, fill='both', expand=True)

tree_DeliveryStaff = ttk.Treeview(deliverystaff_frame, columns=('StaffID', 'First Name', 'Last Name', 'Vehicle Type'))
tree_DeliveryStaff.pack(padx=10, pady=10)

columns_deliverystaff = ('StaffID', 'FirstName', 'LastName', 'VehicleType',)

fetch_delivery_staff_button = tk.Button(deliverystaff_frame, text="Display", command=fetch_DeliveryStaff)
fetch_delivery_staff_button.pack(pady=10)

# Statistics tab--------------------------------------------------------------------------------------------------------------------
statistics_frame = ttk.LabelFrame(tab_statistics, text="Statistics")
statistics_frame.columnconfigure(0, weight=1)
statistics_frame.rowconfigure(0, weight=1)
statistics_frame.pack(padx=10, pady=10, fill='both', expand=True)

mean_rating_button = tk.Button(statistics_frame, text="Calculate Mean Food Rating", command=calculate_mean_rating)
mean_rating_button.pack(pady=10)

histogram_button = tk.Button(statistics_frame, text="Show Delivery Time Histogram", command=draw_histogram)
histogram_button.pack(pady=10)

app.mainloop()